# Práctica II: Uso de modelos Transformers (RAG: Retrieval and Augmented Generation)

## 0. Carga de librerias

In [1]:
import re
import numpy as np
import pandas as pd

import gc
import torch
from itertools import islice

from sentence_transformers import SentenceTransformer
from transformers import pipeline, BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer

from huggingface_hub import list_models, model_info

2025-11-30 00:05:10.292478: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-30 00:05:10.331131: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-30 00:05:11.295099: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# Ignorar errores
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [8]:
# Instalar estas dependencias solo si usas graficas RTX NVIDIA
# !pip install torch==2.5.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# !pip install flash-attn==2.5.8 --no-build-isolation
# !pip install -U bitsandbytes

# Recomendación: ¡Ejecutar desde las celdas 3 (incluidas 2.5.1. la clase 'BuscadorVectoresSmilares', 2.5.2. la clase 'GeneradorChunks' y 2.5.3. la clase 'GeneradorRespuestas')
Porque antes de ellas, solo es para demostrar algunas cosas. Si se quiere ejecutar y ver resultados, mejor hacerlo desde la celda con índice 3 para abajo!

## 1. Corpus de conocimiento
Hemos dejado al lado los videojuegos, y como ingenieros, vamos a enfocarnos en algo más acorde a lo nuestro. Decidimos elegir como fuente **el manueal de BASH.** Este contene casi unas 10k de lineas, por lo que, creemos que es lo suficientemente complejo para probar RAG (a parte que nos gusta el lenguaje BASH):

In [3]:
# Vamos a ver como es el documento
with open('manual_bash.txt', 'r') as bash:

    # Leemos una parte del texto (ejemplo, que es bash)
    for linea in islice(bash, 197, 209):
        print(linea.strip())

1.1 ¿Qué es Bash?
Bash es el intérprete, o el lenguaje intérprete de órdenes, para el sistema operativo gnu.
El nombre es un acrónimo de ‘Bourne-Again SHell’, un juego de palabras con Stephen
Bourne, el autor del ancestro directo del actual intérprete sh, que apareció en la versión
Seventh Edition Bell Labs Research de Unix.
Bash es en gran parte compatible con sh e incorpora funcionalidades útiles del intérprete
Korn ksh y el intérprete C csh. Esta concebido para ser una implementación que se ajusta a
la parte ieee posix Shell and Tools de la especificación ieee posix (ieee Standard 1003.1).
Ofrece mejoras funcionales sobre sh tanto para uso interactivo como para programar.
Aunque el sistema operativo gnu proporciona otros intérpretes de órdenes, incluyen-
do una versión de csh, Bash es el intérprete predeterminado. Al igual que otro software
de gnu, Bash es bastante portable. Actualmente se ejecuta en casi cualquier versión de


vamos a ver cuantas lineas tiene el manual de Bash:

In [4]:
!cat 'manual_bash.txt' | wc -l

9179


Y palabras:

In [5]:
!cat 'manual_bash.txt' | wc -w

98011


Como vemos, es un texto con bastante información. Lo segundo, no solo se explica que es BASH, sino también su lenguaje de programación, consejos, parámetros, etc. Es un documento completo, que trata varias cosa y no solo una. Con todo esto, creemos que es lo suficientemente complejo, al no tratar solamente de algo tipo QA (ejemplo, preguntar que es BASH, para que sirve ls, etc), sino que, veremos si el modelo es capaz de generarnos algún comando según le pidamos lo que queramos. Por todo esto, creemos que el manual de BASH es lo suficientemente complejo para este apartado.

## 2. Preparación del entorno
A diferencia de LoRA, debemos de preparar el entorno a nuestro modelo, dandole la posibilidad de buscar información sobre nuestro tema dándole acceso a este archivo. Prepararemos 2 funciones de búsqueda de similitud de embeddings, probaremos cambiar un poco los chunks, y luego, probaremos con 2 modelos distintos (ejemplo uno normal y otro entrenado para entender código) para ver cuál de los dos sale ganador y por qué.

### 2.1. Carga del generador de embeddings
Hemos cogido 2 modelos, los cuales 1 entiende muy bien el lenguaje español, y el otro, lo mismo, pero con una pequeña ventaja: entiende muy bien el código. Presentamos a ambos modelos:

- **sentence-transformers/paraphrase-multilingual-mpnet-base-v2:** Modelo el cual ha sido entrenado para entender más de 50 idiomas, entre ellos, el español.
- **jinaai/jina-embeddings-v2-base-es:** Como dice su nombre, este modelo fue entrenado con más de 30 lenguajes de programación. A diferencia de mpnet, le cuesta un poco más el español, pero ya veremos como le va.

#### 2.1.1. Carga y uso de MPNET
Vamos a cargar ya el modelo, para tenerlo preparado para generar los embeddings:

In [5]:
# Cargamos el generador de embeddings mpnet
mpnet = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

# Probamos generar una frase
frase_ejemplo = 'BASH es un lenguaje de programacion ideal para los ingenieros'
embedding_ejemplo_mpnet = mpnet.encode(frase_ejemplo)

# Mostramos una parte del embedding
print(f'Primeros 10 valores del embedding: {embedding_ejemplo_mpnet[:10]}')
print(f'Dimension de los embeddings: {embedding_ejemplo_mpnet.shape}')

Primeros 10 valores del embedding: [-0.22837731 -0.10745298 -0.01572983 -0.04717415 -0.02984885  0.01631061
 -0.13659334 -0.04721458  0.11933082  0.08431034]
Dimension de los embeddings: (768,)


#### 2.1.2. Carga y uso de jina
Ahora, con la misma frase, veremos si hay diferencias con jina:

In [ ]:
# Cargamos el generador de embeddings jina
jina = SentenceTransformer('jinaai/jina-embeddings-v2-base-es', trust_remote_code=True, model_kwargs={"dtype": torch.float16})
# Importante el True, sino dara fallos y quepe en GPU

# Codificamos la frase de ejemplo
embedding_ejemplo_jina = jina.encode(frase_ejemplo)

# Mostramos una parte del embedding
print(f'Primeros 10 valores del embedding: {embedding_ejemplo_jina[:10]}')
print(f'Dimension de los embeddings: {embedding_ejemplo_jina.shape}')

Primeros 10 valores del embedding: [-0.02374  -0.01854   0.02446   0.001648 -0.0704   -0.06976  -0.01964
 -0.01174   0.0256    0.0646  ]
Dimension de los embeddings: (768,)


Comparando un poco los embeddings:

In [6]:
print(f'Primeros 5 valores del embedding mpnet: {embedding_ejemplo_mpnet[:5]}')
print(f'Primeros 5 valores del embedding jina: {embedding_ejemplo_jina[:5]}')
print(f'¿Tamaños de embeddings iguales?: {embedding_ejemplo_mpnet.shape == embedding_ejemplo_jina.shape}')

Primeros 5 valores del embedding mpnet: [-0.22837731 -0.10745298 -0.01572983 -0.04717415 -0.02984885]
Primeros 5 valores del embedding jina: [-0.02376082 -0.01850522  0.02447913  0.00156537 -0.07030497]
¿Tamaños de embeddings iguales?: True


Vemos dimensiones de vectores iguales, pero valores distintos. Será interesante ver cuál de ambos modelos embeddings funciona mejor (probando con los 2 modelos a probar).

### 2.2. Búsqueda de embeddings similares
Para ello, hemos decidido probar con 2 de las más usadas: Similitud coseno y la distancia euclideo. Para ello, debemos de preparar una función que calcule dichas distancias. Vamos por partes:

#### 2.2.1. Similitud coseno
Esta ya la vimos en clase, y es la que se suele usar en casi todos los casos. Nos da un valor en el rango [-1, 1], donde 1 es lo más similar y -1, lo más similar, pero al contrario, 0 es que no hay similitud en nada.

In [8]:
# Funcion que calcula la similitud coseno entre 2 vectores
def similitud_coseno(v1, v2):

    # Calculamos el producto punto
    producto_punto = np.dot(v1, v2)

    # Calculamos la longitud de cada vector usando la norma euclidea
    norma_v1 = np.linalg.norm(v1)
    norma_v2 = np.linalg.norm(v2)

    # Devolvemos la similitud (rango -1, 1)
    return producto_punto / (norma_v1 * norma_v2)

# Nota: Reusamos la funcion de clase ya que funciona perfectamente bien

Vamos a probarlo con algunos ejemplo. Usaremos ambos embeddings:

In [9]:
# Reusamos la frase de antes, pero creamos una nueva para comparar
frase_comparar = 'Codigo BASH: echo "Hola, soy ingeniero" | grep ingeniero'
frase_embedding_mpnet = mpnet.encode(frase_comparar)

# Llamamos a la funcion coseno
similitud_coseno_ejemplo_mpnet = similitud_coseno(embedding_ejemplo_mpnet, frase_embedding_mpnet)
print(f'Las frases con mpnet nos dan una similitud coseno: {similitud_coseno_ejemplo_mpnet}')

# Y para jina
frase_embedding_jina = jina.encode(frase_comparar)

# Llamamos a la funcion coseno
similitud_coseno_ejemplo_jina = similitud_coseno(embedding_ejemplo_jina, frase_embedding_jina)
print(f'Las frases con jina nos dan una similitud coseno: {similitud_coseno_ejemplo_jina}')

Las frases con mpnet nos dan una similitud coseno: 0.5496674180030823
Las frases con jina nos dan una similitud coseno: 0.5573885440826416


Es muy interesante, ya que, para ambos modelos, las frases están relacionadas con algo de fuerza, cosa que creemos también nosotros.

#### 2.2.2. Distancia euclidea
Esta es otra que, comparado con la similitud coseno, no se usa casi nada, pero quitando esta, es una de las más usadas. Nos calcula la distancia entre 2 puntos. Veamos como aplicarla:

In [10]:
# Funcion que calcula la distancia euclidea
def distancia_euclidea(v1, v2):

    # Calculamos la distancia euclidea
    distancia_euclidea = np.linalg.norm(v1 - v2)

    # Convertimos en similitud
    similitud_euclidea = 1 / (1 + distancia_euclidea)

    # Acotamos en 0, 1
    return similitud_euclidea

En nuestro caso, 0 es nada similar y 1, lo más similar (igual). Vamos a probarla para cada embedding:

In [11]:
# Reusamos las frases
# Llamamos a la funcion euclidea
distancia_euclidea_ejemplo_mpnet = distancia_euclidea(embedding_ejemplo_mpnet, frase_embedding_mpnet)
print(f'Las frases con mpnet nos dan una distancia euclidea (similitud): {distancia_euclidea_ejemplo_mpnet}')

# Y para jina
frase_embedding_jina = jina.encode(frase_comparar)

# Llamamos a la funcion euclidea
distancia_euclidea_ejemplo_jina = distancia_euclidea(embedding_ejemplo_jina, frase_embedding_jina)
print(f'Las frases con jina nos dan una distancia euclidea (similitud): {distancia_euclidea_ejemplo_jina}')

Las frases con mpnet nos dan una distancia euclidea (similitud): 0.2893986403942108
Las frases con jina nos dan una distancia euclidea (similitud): 0.5152347683906555


Vemos que, aquí la diferencia entre la similitud coseno y euclídea para mpnet es notoria (0.29 ahora, antes 0.55). Para jina, no es 'mucha'. Veremos que pasa probando ambas distancias, con ambos embeddings en la práctica, que a lo mejor nos encontramos con resultados muy interesantes.

## 2.3. Tratamiento del documento
Vamos a cargar el documento, y prepararlo para nuestro modelo. Sabemos que es una pésima idea pasar todo el documento de golpe (límite de tokens,, mucha información procesada, etc), por lo cual, dividirlo en chunks, ayudará mucho al modelo:

In [2]:
# Funcion que limpia el texto para ayudar a los modelos
def limpiar_texto(texto):

    # Limpiamos los espacios en blanco (seguidos)
    texto = re.sub(r'\s+', ' ', texto)

    # Limpiamos los puntos
    texto = re.sub(r'(\.\s*)+', '.', texto)

    return texto

In [3]:
# Funcion que divide nuestro documento en chunks segun le pidamos
def dividir_documento_chunks(documento, len_chunks=300, remember_words=100):

    # Dividimos el texto en palabras
    documento_palabras = documento.replace('\n', '').split()

    # Creamos el chunk
    chunks_totales = list()

    # Recorremos el conjunto de palabras para formar los chunks
    contador = 0
    total_palabras = len(documento_palabras)
    while contador < total_palabras:

        # Calculamos el inicio del chunk
        contador = max(0, contador - remember_words)

        # Calculamos el final del chunk
        limite_chunk = min(contador + len_chunks, total_palabras)
        
        # Sacamos el chunk
        chunk = ' '.join(documento_palabras[contador:limite_chunk])
        chunks_totales.append(chunk)
        
        # Avanzamos el contador dejando 'remember_words' de solapamiento
        contador += len_chunks

    return chunks_totales

En esta función, damos libertad para poner tanto la longitud de los chunks, como cuantas palabras del pasado puede tener los chunks. Recordar que tenemos 98011 palabras, por lo cual, hagamos que los chunks sean acorde al número de palabras:

In [4]:
# Cargamos el documento
documento_bash = None
with open('manual_bash.txt', 'r') as bash:
    documento_bash = bash.read()

# Lo limpiamos
documento_bash_limpio = limpiar_texto(documento_bash)

# Generamos chunks
chunks = dividir_documento_chunks(documento_bash_limpio)

# Mostremos algunos chunks
num_chunks = len(chunks)
print(f'Total de chunks: {num_chunks}')
print(f'Chunk primero: {chunks[0]}')
print(f'Chunk aleatorio: {chunks[np.random.randint(0, num_chunks - 1, 1)[0]]}')
print(f'Ultimo chunk: {chunks[-1]}')

Total de chunks: 378
Chunk primero: Manual de Referencia de Bash Documentación de Referencia para Bash Edición 4.4, para Bash Versión 4.4.Septiembre de 2016 Chet Ramey, Case Western Reserve University Brian Fox, Free Software Foundation.Traducido por Jorge Maldonado.Revisado por David Arroyo Menéndez.Este texto es una breve descripción de las funcionalidades presentes en el intérprete de órdenes de Bash (version 4.4, 7 de septiembre de 2016).Esta es la Edición 4.4, actualizada por última vez 7 de septiembre de 2016, de Manual de Referencia de Bash, para Bash, Version 4.4.Las fuentes de este documento están accesibles desde https://github.com/davidam/bashrefes.git Copyright c 1988–2016 Free Software Foundation, Inc.Permission is granted to copy, distribute and/or modify this document under the terms of the GNU Free Documentation License, Version 1.3 or any later ver- sion published by the Free Software Foundation; with no Invariant Sections, no Front-Cover Texts, and no Back-C

Como ya tenemos los chunks, vamos a ver como funciona el hacerle embedding con nuestros modelos:

In [16]:
# Sacamos una frase (ejemplo el 5to)
chunk_ejemplo = chunks[4]  # Explica que es bash

# Hacemos una pregunta de bash
pregunta_bash = '¿Que es bash? ¿Que es un interprete?'

# Hacemos embeddings con mpnet
embedding_chunk_mpnet = mpnet.encode(chunk_ejemplo)
embedding_pregunta_mpnet = mpnet.encode(pregunta_bash)

# Mostramos la similitud
similitud_coseno_mpnet = similitud_coseno(embedding_chunk_mpnet, embedding_pregunta_mpnet)
similitud_euclidea_mpnet = distancia_euclidea(embedding_chunk_mpnet, embedding_pregunta_mpnet)
print('=' * 12, 'Para MPNET', '=' * 12)
print(f'Similitud coseno: {similitud_coseno_mpnet}')
print(f'Similitud euclidea: {similitud_euclidea_mpnet}')

# Ahora para jina
# Hacemos embeddings con jina
embedding_chunk_jina = jina.encode(chunk_ejemplo)
embedding_pregunta_jina = jina.encode(pregunta_bash)

# Mostramos la similitud
similitud_coseno_jina = similitud_coseno(embedding_chunk_jina, embedding_pregunta_jina)
similitud_euclidea_jina = distancia_euclidea(embedding_chunk_jina, embedding_pregunta_jina)
print('=' * 12, 'Para JINA', '=' * 12)
print(f'Similitud coseno: {similitud_coseno_jina}')
print(f'Similitud euclidea: {similitud_euclidea_jina}')

============ Para MPNET ============
Similitud coseno: 0.3404637277126312
Similitud euclidea: 0.2449530065059662
============ Para JINA ============
Similitud coseno: 0.7231422066688538
Similitud euclidea: 0.5733548402786255


Con un chunk de 300 palabras, creemos que esta bien. Por si las moscas, probaremos con un chunk de 500 (no más por no querer pasar los límites de tokens, y hacer sufrir al modelo)

## 2.4. Generar embeddings de los chunks
Como último paso, vamos a 'tokenizar' los chunks con ambos modelos:

## 2.4.1. Embeddings con MPNET

In [29]:
# Le decimos a MPNET que nos genere los embeddings
embeddings_mpnet = mpnet.encode(chunks)

# Mostramos algunos
print(f'Total de embeddings*dimension de los embeddings: {embeddings_mpnet.shape}')
print(f'Primer embedding (primeros 10 valores): {embeddings_mpnet[0][:5]}')

Total de embeddings*dimension de los embeddings: (378, 768)
Primer embedding (primeros 10 valores): [-0.17171276 -0.13587809 -0.00456395 -0.00232998  0.05197524]


## 2.4.2. Embeddings con JINA

In [7]:
# Le decimos a JINA que nos genere los embeddings
embeddings_jina = jina.encode(chunks)

# Mostramos algunos
print(f'Total de embeddings*dimension de los embeddings: {embeddings_jina.shape}')
print(f'Primer embedding (primeros 10 valores): {embeddings_jina[0][:5]}')

Total de embeddings*dimension de los embeddings: (378, 768)
Primer embedding (primeros 10 valores): [-0.03308  -0.0771   -0.01063  -0.005733 -0.05685 ]


## 2.5. Preparando todo para ejecutar todas las opciones
Hasta ahora, hemos visto paso a paso todo el proceso de RAG (falta solo la obtención de mejores embeddings). Ahora, vamos a automatizar todo para que las pruebas sean más sencillas. Daremos opciones para que los modelos a probar pueda buscar similitudes por distancia coseno o euclídea, y dejaremos todo preparado para que con solo cambiar algunos parámetros, podamos probar todas las opciones posible:

### 2.5.1. Definiendo nuestra función de búsqueda de chunks similares
Definimos nuestra función que se encargará de devolvernos los chunks más 'similares' al de nuestra pregunta (con ayuda del generador de embeddings):

In [2]:
class BuscadorVectoresSmilares:

    def __init__(self, chunks, encoding_chunks, pregunta, encoding_pregunta, coseno=True, top_k=10):
        self.__chunks = chunks
        self.__encoding_chunks = encoding_chunks
        self.__pregunta = pregunta
        self.__encoding_pregunta = encoding_pregunta
        
        self.__funcion_similitud = self.__similitud_coseno if coseno else self.__similitud_euclidea
        self.__top_k = top_k


    # Funcion que devuelve los 'k' chunks similares a la pregunta
    def obtener_chunks_similares(self):

        # Definimos una lista con las similitudes y chunks
        lista_similitudes = list()

        # Recorremos cada chunk para ver las similitudes
        i = 0
        for chunk, encoding_chunk in zip(self.__chunks, self.__encoding_chunks):

            # Calculamos su distancia
            similitud_pregunta = self.__funcion_similitud(self.__encoding_pregunta, encoding_chunk)

            # Lo guardamos
            informacion_chunk = {'num_chunk': i, 'similitud': similitud_pregunta, 'chunk': chunk}
            lista_similitudes.append(informacion_chunk)
            i += 1

        # Ordenamos los chunks de mayor a menor similitud
        lista_similitudes = sorted(lista_similitudes, key=lambda x: x['similitud'], reverse=True)

        # Devolvemos los 'k' chunks mas similares
        return lista_similitudes[:self.__top_k]


    # Funcion que calcula la distancia euclidea
    def __similitud_euclidea(self, v1, v2):

        # Calculamos la distancia euclidea
        distancia_euclidea = np.linalg.norm(v1 - v2)

        # Convertimos en similitud
        similitud_euclidea = 1 / (1 + distancia_euclidea)

        # Acotamos en 0, 1
        return similitud_euclidea


    # Funcion que calcula la similitud coseno entre 2 vectores
    def __similitud_coseno(self, v1, v2):

        # Calculamos el producto punto
        producto_punto = np.dot(v1, v2)

         # Calculamos la longitud de cada vector usando la norma euclidea
        norma_v1 = np.linalg.norm(v1)
        norma_v2 = np.linalg.norm(v2)

        # Devolvemos la similitud (rango -1, 1)
        return producto_punto / (norma_v1 * norma_v2)

Lo probamos:

In [13]:
# Definimos el top k
top_k = 3

# Probamos para similitud coseno y euclidea
for use_cosine in [True, False]:

    # Mostramos los resultados
    print('=' * 12, f'Usando similitud coseno: {False}', '=' * 12)
    buscador_similitudes = BuscadorVectoresSmilares(chunks, embeddings_jina, pregunta_bash, embedding_pregunta_jina, False, top_k)
    vectores_similares = buscador_similitudes.obtener_chunks_similares()
    
    print(f'Obteniendo los {top_k} vectores mas similares:')
    print(f'\t- Pregunta: {pregunta_bash}')
    
    # Mostramos cada chunk
    for info_chunk in vectores_similares:
    
        print('-' * 56)
        print(f'\tNumero de chunk: {info_chunk["num_chunk"]}')
        print(f'\tSimilitud: {info_chunk["similitud"]}')
        print(f'\tParte del chunk: {info_chunk["chunk"][:25]}')

============ Usando similitud coseno: False ============
Obteniendo los 3 vectores mas similares:
	- Pregunta: ¿Que es bash? ¿Que es un interprete?
--------------------------------------------------------
	Numero de chunk: 4
	Similitud: 0.5732421875
	Parte del chunk: .153 iv 10 Instalación d
--------------------------------------------------------
	Numero de chunk: 5
	Similitud: 0.5576171875
	Parte del chunk: para ser una implementaci
--------------------------------------------------------
	Numero de chunk: 7
	Similitud: 0.5400390625
	Parte del chunk: instrucciones es esencial
============ Usando similitud coseno: False ============
Obteniendo los 3 vectores mas similares:
	- Pregunta: ¿Que es bash? ¿Que es un interprete?
--------------------------------------------------------
	Numero de chunk: 4
	Similitud: 0.5732421875
	Parte del chunk: .153 iv 10 Instalación d
--------------------------------------------------------
	Numero de chunk: 5
	Similitud: 0.5576171875
	Parte del chunk: 

Es de destacar que se eligen los mismos chunks con cualquier distancia, luego veremos si hay diferencias o no.

### 2.5.2. Definiendo nuestra función de obtención de chunks
Automatizamos esto para que el cambio de chunks sea más fácil:

In [3]:
class GeneradorChunks:

    def __init__(self, documento, len_chunks=300, remember_words=100):
        self.__documento = self.__limpiar_texto(documento)
        self.__len_chunks = len_chunks
        self.__remember_words = remember_words

    
    # Funcion que divide nuestro documento en chunks segun le pidamos
    def dividir_documento_chunks(self):

        # Dividimos el texto en palabras
        documento = self.__documento.replace('\n', '').split()

        # Creamos el chunk
        chunks_totales = list()

        # Recorremos el conjunto de palabras para formar los chunks
        contador = 0
        total_palabras = len(documento)
        while contador < total_palabras:

            # Calculamos el inicio del chunk
            contador = max(0, contador - self.__remember_words)

            # Calculamos el final del chunk
            limite_chunk = min(contador + self.__len_chunks, total_palabras)
            
            # Sacamos el chunk
            chunk = ' '.join(documento[contador:limite_chunk])
            chunks_totales.append(chunk)
            
            # Avanzamos el contador dejando 'remember_words' de solapamiento
            contador += self.__len_chunks

        return chunks_totales

    
    # Funcion que limpia el texto para ayudar a los modelos
    def __limpiar_texto(self, texto):

        # Limpiamos los espacios en blanco (seguidos)
        texto = re.sub(r'\s+', ' ', texto)

        # Limpiamos los puntos
        texto = re.sub(r'(\.\s*)+', '.', texto)

        return texto

Probamos:

In [34]:
generador_chunks = GeneradorChunks(documento_bash)
generador_chunks.dividir_documento_chunks()[:3]

['Manual de Referencia de Bash Documentación de Referencia para Bash Edición 4.4, para Bash Versión 4.4.Septiembre de 2016 Chet Ramey, Case Western Reserve University Brian Fox, Free Software Foundation.Traducido por Jorge Maldonado.Revisado por David Arroyo Menéndez.Este texto es una breve descripción de las funcionalidades presentes en el intérprete de órdenes de Bash (version 4.4, 7 de septiembre de 2016).Esta es la Edición 4.4, actualizada por última vez 7 de septiembre de 2016, de Manual de Referencia de Bash, para Bash, Version 4.4.Las fuentes de este documento están accesibles desde https://github.com/davidam/bashrefes.git Copyright c 1988–2016 Free Software Foundation, Inc.Permission is granted to copy, distribute and/or modify this document under the terms of the GNU Free Documentation License, Version 1.3 or any later ver- sion published by the Free Software Foundation; with no Invariant Sections, no Front-Cover Texts, and no Back-Cover Texts.A copy of the license i

### 2.5.3. Definiendo nuestra función de pruebas
Con esto, con pasarle el modelo generador, el documento, la pregunta, el modelo de embeddings y las configuraciones (similitudes, tamaños del chunk, etc), podemos generar respuestas fácilmente:

In [4]:
class GeneradorRespuestas:

    def __init__(self, generador, nombre_generador, generador_embeddings, nombre_generador_embeddings, pregunta, documento,
                    params_generador, params_chunks, params_similitud):
        self.__generador = generador
        self.__nombre_generador = nombre_generador

        self.__generador_embeddings = generador_embeddings
        self.__nombre_gemerador_embeddings = nombre_generador_embeddings

        self.__pregunta = pregunta
        self.__documento = documento

        self.__params_generador = params_generador
        self.__params_chunks = params_chunks
        self.__params_similitud = params_similitud

        self.__prompt = None


    # Funcion que genera los embeddings
    def __generar_embeddings(self, chunks):

        # Generamos los embeddings de los chunks
        embeddings_chunk = self.__generador_embeddings.encode(chunks, convert_to_tensor=True, device="cuda").cpu()

        # Generamos los embeddings de la pregunta
        embeddings_pregunta = self.__generador_embeddings.encode(self.__pregunta, convert_to_tensor=True, device="cuda").cpu()

        # Devolvemos ambos
        return embeddings_pregunta, embeddings_chunk

    
    # Funcion que obtiene los chunks similares
    def __obtener_chunks_similares(self, chunks):

        # Obtenemos los embeddings
        embedding_pregunta, embedding_chunks = self.__generar_embeddings(chunks)

        # Iniciamos nuestro buscador de chunks similares
        buscador_chunks = BuscadorVectoresSmilares(chunks, embedding_chunks, self.__pregunta, embedding_pregunta, **self.__params_similitud)

        # Hacemos la busqueda de vectores
        return buscador_chunks.obtener_chunks_similares()


    # Funcion que divide el texto en chunks
    def __obtener_chunks(self):

        # Creamos nuestro generador de chunks
        generador_chunks = GeneradorChunks(self.__documento, **self.__params_chunks)

        # Generamos los chunks
        return generador_chunks.dividir_documento_chunks()


    # Funcion que genera el prompt
    def __generar_prompt(self, pregunta, chunks_ayuda):

        # Concatenamos los chunks
        chunks_string = '\n\n'.join([f'{chunk["num_chunk"]}.- {chunk["chunk"]}' for chunk in chunks_ayuda])

        # Definimos nuestro prompt (vamos a meterle un rol para que sepa mejor que hacer)
        prompt = f'''
Eres un asistente de IA experto, y sabes algo de BASH.

Tu trabajo es responder preguntas, tomando de base ÚNICAMENTE la información de los fragmentos proporcionados.

INSTRUCCIONES:
- Si la información está en los fragmentos, responde de forma clara y detallada
- Si NO está en los fragmentos, di: "No encuentro esa información en los fragmentos"
- No inventes información que no esté en los fragmentos
- Es fundamental que digas que fragmentos has usados para tener mejor credibilidad (ejemplo: [Contexto 1], [Contexto 2], etc). No es necesario que
muestres los fragmentos, con citar el numero del fragmento, es suficiente.
__________________________________________________
Contexto necesario para responder
__________________________________________________
{chunks_string}
__________________________________________________
Fin de contexto
__________________________________________________
Pregunta: {pregunta}
'''
        # Lo devolvemos
        return prompt


    # Funcion que muestra la respuesta del modelo por pantalla
    def __mostrar_respuesta_modelo(self, respuesta):

        # Mostramos los datos usados a la hora de configurar todo
        print('=' * 10, f'Resultados con {self.__nombre_generador}', '=' * 10)
        print('-' * 10, 'Configuraciones:', '-' * 10)
        print(f'\t- Nombre del modelo generador: {self.__nombre_generador}')
        print(f'\t- Nombre del modelo de embeddings: {self.__nombre_gemerador_embeddings}')
        print(f'\t- Especificaciones del modelo generador: {self.__params_generador}')
        print(f'\t- Especificaciones de los chunks: {self.__params_chunks}')
        print(f'\t- Especificaciones de la busqueda de similitud: {self.__params_similitud}')
        print(f'-' * 25)
        print(f'Pregunta hecha al modelo: {self.__pregunta}')
        print(f'Respuesta: {respuesta[len(self.__prompt):].strip()}')
        print('=' * 50)

    
    # Funcion que hace todo el proceso de generacion de respuestas
    def generar_respueta(self):

        # Obtenemos los chunks del documento
        chunks_doc = self.__obtener_chunks()

        # Obtenemos los chunks más importantes
        chunks_relevantes = self.__obtener_chunks_similares(chunks_doc)

        # Generamos el prompt con los resultados
        self.__prompt = self.__generar_prompt(self.__pregunta, chunks_relevantes)

        # Liberamos memoria
        del chunks_relevantes
        gc.collect()
        torch.cuda.empty_cache()

        # Se lo pasamos al modelo
        respuesta = self.__generador(
            self.__prompt,
            num_return_sequences=1,
            **self.__params_generador
        )[0]['generated_text']

        # Mostramos la respuesta
        self.__mostrar_respuesta_modelo(respuesta)

    
    # Funcion que permite ver el prompt si se ha generado una respuesta
    def mostrar_prompt(self):

        # Vemos si se ha generado algo
        if self.__prompt is None:
            print('No se ha generado una respuesta. Pregunta algo sin miedo :)')

        else:
            print(f'Prompt generado para la respuesta:\n{self.__prompt}')

Con esto, solo es cambiar lo que le pasamos a la clase, y se hará más fácil probar diferentes combinaciones

## 3. Generando respuestas
Al tener todo ya preparado, solo nos queda probar 2 modelos decoders de generación de texto, y ver como les va:

### 3.1. Obtención de dichos modelos
Vamos a usar nuestro querido list_models para obtener los mejores modelos de generación de texto:

In [7]:
# Reusamos la funcion de lora para mostrar informacion de los modelos
def obtener_informacion_modelo(id_modelo):

    # Obtenemos la informacion del modelo
    info_modelo = model_info(id_modelo)

    # Preparamos algunos datos
    card_data = getattr(info_modelo, 'card_data', 'None')
    licencia = card_data['license'] if card_data is not None else 'None'

    # Organizamos la informacion en un dict
    informacion = {
        'nombre': getattr(info_modelo, 'id', 'None'),
        'autor': getattr(info_modelo, 'author', 'None'),
        'tarea': getattr(info_modelo, 'pipeline_tag', 'None'),
        'fecha-creacion': getattr(info_modelo, 'created_at', 'None'),
        'fecha-ultima-acualizacion': getattr(info_modelo, 'last_modified', 'None'),
        'descargas': getattr(info_modelo, 'downloads', 'None'),
        'likes': getattr(info_modelo, 'likes', 'None'),
        'tags': ', '.join(getattr(info_modelo, 'tags', 'None'),),
        'licencia': licencia
    }

    return informacion 

In [7]:
# Buscamos modelos text-generation
modelos_decoder_normal = list_models(
    filter=['text-generation'],
    sort='downloads',
    direction=-1,
    limit=20
)

# Convertimos a lista y descartamos los que tengan "llama" en el nombre (por cosa de licencia y eso)
modelos_filtrados_normal = [m.id.lower() for m in modelos_decoder_normal if "llama" not in m.modelId.lower()]

# Los mostramos en formato dataframe
informacion_modelos_decoder_normal = [obtener_informacion_modelo(id_modelo) for id_modelo in modelos_filtrados_normal]
info_dataframe_normal = pd.DataFrame(informacion_modelos_decoder_normal)
info_dataframe_normal

,nombre,autor,tarea,fecha-creacion,fecha-ultima-acualizacion,descargas,likes,tags,licencia
0,openai-community/gpt2,openai-community,text-generation,2022-03-02 23:29:04+00:00,2024-02-19 10:57:45+00:00,9667886,3040,"transformers, pytorch, tf, jax, tflite, rust, ...",mit
1,Qwen/Qwen2.5-7B-Instruct,Qwen,text-generation,2024-09-16 11:55:40+00:00,2025-01-12 02:10:10+00:00,7679236,909,"transformers, safetensors, qwen2, text-generat...",apache-2.0
2,openai/gpt-oss-20b,openai,text-generation,2025-08-04 22:33:29+00:00,2025-08-26 17:25:47+00:00,7149808,3995,"transformers, safetensors, gpt_oss, text-gener...",apache-2.0
3,Qwen/Qwen3-0.6B,Qwen,text-generation,2025-04-27 03:40:08+00:00,2025-07-26 03:46:27+00:00,6896153,823,"transformers, safetensors, qwen3, text-generat...",apache-2.0
4,Qwen/Qwen2.5-3B-Instruct,Qwen,text-generation,2024-09-17 14:08:52+00:00,2024-09-25 12:33:00+00:00,6511037,334,"transformers, safetensors, qwen2, text-generat...",other
5,Qwen/Qwen3-4B-Instruct-2507,Qwen,text-generation,2025-08-05 10:58:03+00:00,2025-09-17 06:56:53+00:00,5298747,504,"transformers, safetensors, qwen3, text-generat...",apache-2.0
6,Qwen/Qwen2.5-1.5B-Instruct,Qwen,text-generation,2024-09-17 14:10:29+00:00,2024-09-25 12:32:50+00:00,5184969,558,"transformers, safetensors, qwen2, text-generat...",apache-2.0
7,Qwen/Qwen3-Embedding-0.6B,Qwen,feature-extraction,2025-06-03 14:25:32+00:00,2025-06-20 09:31:05+00:00,5151924,754,"sentence-transformers, safetensors, qwen3, tex...",apache-2.0
8,dphn/dolphin-2.9.1-yi-1.5-34b,dphn,text-generation,2024-05-18 04:50:56+00:00,2025-09-08 05:56:56+00:00,4671896,51,"transformers, safetensors, llama, text-generat...",apache-2.0
9,Qwen/Qwen3-8B,Qwen,text-generation,2025-04-27 03:42:21+00:00,2025-07-26 03:49:13+00:00,4584478,776,"transformers, safetensors, qwen3, text-generat...",apache-2.0


Usaremos 'Qwen/Qwen2.5-1.5B-Instruct' como primer modelo (sabe el lenguaje español al ser multilingual, encima que no pesa mucho). Esto porque estuvimos buscando, pero si ponemos 'es' de filtro, no salen muchos modelos que pueden ser muy útiles.

Ahora, veamos tambien modelos especializados en código:

In [9]:
# Buscamos modelos text-generation
modelos_decoder_especial = list_models(
    filter=['text-generation', 'code'],
    sort='downloads',
    direction=-1,
    limit=40
)

# Convertimos a lista y descartamos los que no sean de qwen (para variar)
modelos_filtrados_especial = [m.id for m in modelos_decoder_especial if "qwen" not in m.modelId.lower()]

# Los mostramos en formato dataframe
informacion_modelos_decoder_especial = [obtener_informacion_modelo(id_modelo) for id_modelo in modelos_filtrados_especial]
info_dataframe_especial = pd.DataFrame(informacion_modelos_decoder_especial)
info_dataframe_especial

,nombre,autor,tarea,fecha-creacion,fecha-ultima-acualizacion,descargas,likes,tags,licencia
0,bigscience/bloomz-560m,bigscience,text-generation,2022-10-08 16:14:42+00:00,2023-05-27 17:27:11+00:00,1793158,129,"transformers, pytorch, tensorboard, safetensor...",bigscience-bloom-rail-1.0
1,microsoft/Phi-3-mini-4k-instruct,microsoft,text-generation,2024-04-22 16:18:17+00:00,2024-09-20 18:09:38+00:00,1275591,1338,"transformers, safetensors, phi3, text-generati...",mit
2,microsoft/phi-2,microsoft,text-generation,2023-12-13 21:19:59+00:00,2024-04-29 16:25:56+00:00,924026,3410,"transformers, safetensors, phi, text-generatio...",mit
3,microsoft/phi-4,microsoft,text-generation,2024-12-11 11:47:29+00:00,2025-11-24 16:57:37+00:00,464995,2190,"transformers, safetensors, phi3, text-generati...",mit
4,microsoft/Phi-3.5-vision-instruct,microsoft,image-text-to-text,2024-08-16 23:48:22+00:00,2024-09-26 22:42:52+00:00,437560,715,"transformers, safetensors, phi3_v, text-genera...",mit
5,microsoft/Phi-4-multimodal-instruct,microsoft,automatic-speech-recognition,2025-02-24 22:33:32+00:00,2025-05-01 15:26:55+00:00,408144,1540,"transformers, safetensors, phi4mm, text-genera...",mit
6,BSC-LT/ALIA-40b,BSC-LT,text-generation,2024-12-09 14:04:29+00:00,2025-10-22 14:25:36+00:00,361024,86,"transformers, safetensors, llama, text-generat...",apache-2.0
7,microsoft/Phi-3-mini-128k-instruct,microsoft,text-generation,2024-04-22 16:26:23+00:00,2025-03-02 22:28:37+00:00,323052,1682,"transformers, safetensors, phi3, text-generati...",mit
8,microsoft/Phi-3.5-mini-instruct,microsoft,text-generation,2024-08-16 20:48:26+00:00,2025-03-02 22:27:58+00:00,308906,930,"transformers, safetensors, phi3, text-generati...",mit
9,microsoft/Phi-4-mini-instruct,microsoft,text-generation,2025-02-19 01:00:58+00:00,2025-05-01 15:27:30+00:00,291935,635,"transformers, safetensors, phi3, text-generati...",mit


Para este caso, nos quedamos con 'microsoft/Phi-3.5-mini-instruct', ya que es el que menos pesa (con algunos peros, ya que cuantizamos).

Dato curioso: Ambos modelos no están marcados como 'text-generation', pero al ser recientes, vimos que no se les clasificó bien:
- A veces en Hugging Face, especialmente con modelos nuevos o subidos por usuarios (como context-labs), las etiquetas de "Task" se ponen mal o se asignan automáticamente de forma extraña.

### 3.2. Preparación de las configuraciones
Antes, vamos a tener a mano todo lo que necesitamos para ir probando (separamos en celdas para ir haciendo pruebas, sino nos quedamos sin VRAM):

In [6]:
# Recordemos que tenemos los embeddings: mpnet
nombre_mpnet = 'paraphrase-multilingual-mpnet-base-v2'
mpnet = SentenceTransformer(nombre_mpnet)

In [6]:
# Recordemos que tenemos los embeddings: jina
nombre_jina = 'jinaai/jina-embeddings-v2-base-es'
jina = SentenceTransformer(nombre_jina, trust_remote_code=True, model_kwargs={"dtype": torch.float16}, device="cpu")  # Para que no ocupe espacio en GPU

In [7]:
# Tambien tenemos el documento leido
documento_bash = None
with open('manual_bash.txt', 'r') as bash:
    documento_bash = bash.read()

In [8]:
# Carguemos los modelos generativos a probar: qwen
nombre_qwen = 'Qwen/Qwen2.5-1.5B-Instruct'
qwen = pipeline(
    'text-generation',# prueba_uno.mostrar_prompt()
    model=nombre_qwen,
    dtype= torch.float16,
    device=0
)

Device set to use cuda:0


In [8]:
# Carguemos los modelos generativos a probar: phi
nombre_phi = "microsoft/Phi-3.5-mini-instruct"

# Cuantización en 4-bit para reducir VRAM (sino nos explota)
phi_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16"
)

# Sacamos su tokenizador (lo que en verdad nos importa)
token_phi = AutoTokenizer.from_pretrained(nombre_phi)

# Y el modelo
model_phi = AutoModelForCausalLM.from_pretrained(
    nombre_phi,
    quantization_config=phi_config,
    device_map="auto"   # reparte capas entre GPU y CPU
)

# Dejamos el pipeline preparado
phi = pipeline(
    "text-generation",
    model=model_phi,
    tokenizer=token_phi,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


No queriamos cuantizar para ver todo el potencial, pero no aguanta nuestra gráfica.

### 3.3. Hora de probar los modelos
Es la hora de divertirnos y ver los resultados de los modelos.

In [8]:
# Preguntas con la cual comenzaremos
pregunta_normal = '¿Que es un interprete? ¿Cuales son sus funcionalidades basicas? Responde en menos de 50 palabras'
pregunta_codigo = '¿Para que sirve el comando "cd"? ¿Puedes poner un ejemplo? Responde en menos de 50 palabras'

#### 3.3.1. Pruenas para qwen y mpnet
Vamos a realizar algunas pruebas con qwen y mpnet (los modelos normales).

##### 3.3.1.1. Probando con similitud coseno, con mpnet (lenguaje español sin especializar), con chunk de tamaño 300, remember de 100 y top_k=10
Esta sería como la prueba estándar, para ver si nos va bien o no:

In [16]:
# Generamos los parametros de las clases
params_generador = {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
params_chunks = {'len_chunks': 300, 'remember_words': 100}
params_similitud = {'coseno': True, 'top_k': 10}

# Instanciamos la clase de pruebas
prueba_uno = GeneradorRespuestas(qwen, nombre_qwen, mpnet, nombre_mpnet, pregunta_normal, documento_bash, params_generador, params_chunks, params_similitud)

# Mostramos el resultado
prueba_uno.generar_respueta()

/tmp/ipykernel_242387/3760732647.py:55: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  producto_punto = np.dot(v1, v2)


========== Resultados con Qwen/Qwen2.5-1.5B-Instruct ==========
---------- Configuraciones: ----------
	- Nombre del modelo generador: Qwen/Qwen2.5-1.5B-Instruct
	- Nombre del modelo de embeddings: paraphrase-multilingual-mpnet-base-v2
	- Especificaciones del modelo generador: {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
	- Especificaciones de los chunks: {'len_chunks': 300, 'remember_words': 100}
	- Especificaciones de la busqueda de similitud: {'coseno': True, 'top_k': 10}
-------------------------
Pregunta hecha al modelo: ¿Que es un interprete? ¿Cuales son sus funcionalidades basicas? Responde en menos de 50 palabras
Respuesta: Respuesta:

Un interpretador es un software que lee y ejecuta instrucciones programadas por un humano. Sus características básicas incluyen realizar operaciones basadas en el código fuente ingresado, manejar diferentes tipos de instrucciones y permear entre ellas para hacer cumplir un programa. 

Estos son algunos de sus princip

Miremos el prompt:

In [12]:
# prueba_uno.mostrar_prompt()

Vemos que lo hace bastante bien. De hecho, esta muy bien explicado, aunque no nos dice de donde saca los chunks. Un 8/10 (por lo de los chunks).

##### 3.3.1.2. Probando con similitud euclidea, con mpnet (lenguaje español sin especializar), con chunk de tamaño 300, remember de 100 y top_k=10
Vamos a ver como le va con la distancia euclidea:

In [13]:
# Generamos los parametros de las clases
params_generador = {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
params_chunks = {'len_chunks': 300, 'remember_words': 100}
params_similitud = {'coseno': False, 'top_k': 10}

# Instanciamos la clase de pruebas
prueba_dos = GeneradorRespuestas(qwen, nombre_qwen, mpnet, nombre_mpnet, pregunta_normal, documento_bash, params_generador, params_chunks, params_similitud)

# Mostramos el resultado
prueba_dos.generar_respueta()

========== Resultados con Qwen/Qwen2.5-1.5B-Instruct ==========
---------- Configuraciones: ----------
	- Nombre del modelo generador: Qwen/Qwen2.5-1.5B-Instruct
	- Nombre del modelo de embeddings: paraphrase-multilingual-mpnet-base-v2
	- Especificaciones del modelo generador: {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
	- Especificaciones de los chunks: {'len_chunks': 300, 'remember_words': 100}
	- Especificaciones de la busqueda de similitud: {'coseno': False, 'top_k': 10}
-------------------------
Pregunta hecha al modelo: ¿Que es un interprete? ¿Cuales son sus funcionalidades basicas? Responde en menos de 50 palabras
Respuesta: Respuesta:

Un interpretador es un programa que lee y ejecuta código fuente en tiempo real, similar al comportamiento de un compilador, pero generalmente para scripts y programas muy cortos. Sus principales características y funcionalidades incluyen:

• EJECUCIÓN DE CÓDIGO FUENTE INFINITAS: Ejecuta instrucciones repetidamentes 

Miremos el prompt:

In [14]:
# prueba_dos.mostrar_prompt()

Vemos que, no se queda atrás. Incluso puede parecer que lo hace mejor, pero falla en nombrar las fuentes, igual que el coseno (y una linea rara que generó). Un 8/10 para nosotros.

##### 3.3.1.3. Probando con similitud coseno, con mpnet (lenguaje español sin especializar), con chunk de tamaño 500, remember de 200 y top_k=20
Aumentamos los parámetros del chunk, a ver si cambia algo la cosa

In [39]:
# Generamos los parametros de las clases
params_generador = {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}  # Preferimos al final no moverlo porque funciona bastante bien
params_chunks = {'len_chunks': 500, 'remember_words': 200}
params_similitud = {'coseno': True, 'top_k': 20}

# Instanciamos la clase de pruebas
prueba_tres = GeneradorRespuestas(qwen, nombre_qwen, mpnet, nombre_mpnet, pregunta_normal, documento_bash, params_generador, params_chunks, params_similitud)

# Mostramos el resultado
prueba_tres.generar_respueta()

/tmp/ipykernel_242387/3760732647.py:55: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  producto_punto = np.dot(v1, v2)


========== Resultados con Qwen/Qwen2.5-1.5B-Instruct ==========
---------- Configuraciones: ----------
	- Nombre del modelo generador: Qwen/Qwen2.5-1.5B-Instruct
	- Nombre del modelo de embeddings: paraphrase-multilingual-mpnet-base-v2
	- Especificaciones del modelo generador: {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
	- Especificaciones de los chunks: {'len_chunks': 500, 'remember_words': 200}
	- Especificaciones de la busqueda de similitud: {'coseno': True, 'top_k': 20}
-------------------------
Pregunta hecha al modelo: ¿Que es un interprete? ¿Cuales son sus funcionalidades basicas? Responde en menos de 50 palabras
Respuesta: Resposta: Un interprete es un procesador de macros que ejecuta instrucciones. Su función básica es ejecutar scripts escritos en lenguajes de scripting, proporcionando así la interface con el sistema operativo. Funcionalidades básicas incluyen manejo de entrada/salida, redirecciones, traps, shell built-ins, etc. [Mostrar contexto

Al parecer, con mucha información para los chunks, explota. Es mejor no poner muchos chunks.

##### 3.3.1.4. Probando con similitud euclidea, con mpnet (lenguaje español sin especializar), con chunk de tamaño 500, remember de 200 y top_k=20
Lo mismo que antes, cambiando solo la similitud:

In [21]:
# Generamos los parametros de las clases
params_generador = {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}  # Preferimos al final no moverlo porque funciona bastante bien
params_chunks = {'len_chunks': 500, 'remember_words': 200}
params_similitud = {'coseno': False, 'top_k': 20}

# Instanciamos la clase de pruebas
prueba_cuatro = GeneradorRespuestas(qwen, nombre_qwen, mpnet, nombre_mpnet, pregunta_normal, documento_bash, params_generador, params_chunks, params_similitud)

# Mostramos el resultado
prueba_cuatro.generar_respueta()

========== Resultados con Qwen/Qwen2.5-1.5B-Instruct ==========
---------- Configuraciones: ----------
	- Nombre del modelo generador: Qwen/Qwen2.5-1.5B-Instruct
	- Nombre del modelo de embeddings: paraphrase-multilingual-mpnet-base-v2
	- Especificaciones del modelo generador: {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
	- Especificaciones de los chunks: {'len_chunks': 500, 'remember_words': 200}
	- Especificaciones de la busqueda de similitud: {'coseno': False, 'top_k': 20}
-------------------------
Pregunta hecha al modelo: ¿Que es un interprete? ¿Cuales son sus funcionalidades basicas? Responde en menos de 50 palabras
Respuesta: Respuesta: Un interpretador es un entorno virtualizado que lee y ejecuta instrucciones, con las capacidades básicas de interactuar con otros programas y almacenar y acceder a datos. Funciones como ejecutar scripts, ver informacion del sistema, ejecutar instrucciones de la shell, traer ayuda sobre el uso de los comandos, entre o

Lo hace muy bien, solo que no se explica mucho. Esta es una opción viable, pero creemos que se puede explicar mejor.

##### 3.3.1.5. Conclusiones pregunta normal
Luego de probar 2 modelos (uno generador y otro de embeddings), podemos decir que lo hacen bien para preguntas normales. Sin embargo, no hemos probado todavía preguntas específicas con algo de código. Usaremos las mejores configuraciones de cada similitud para ver cual de ambas funciona mejor para preguntas más 'especiales':

- Para la similitud coseno:

In [33]:
# Generamos los parametros de las clases
params_generador = {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
params_chunks = {'len_chunks': 300, 'remember_words': 100}
params_similitud = {'coseno': True, 'top_k': 10}

# Instanciamos la clase de pruebas
prueba_codigo_coseno = GeneradorRespuestas(qwen, nombre_qwen, mpnet, nombre_mpnet, pregunta_codigo, documento_bash, params_generador, params_chunks, params_similitud)

# Mostramos el resultado
prueba_codigo_coseno.generar_respueta()

/tmp/ipykernel_242387/3760732647.py:55: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  producto_punto = np.dot(v1, v2)


========== Resultados con Qwen/Qwen2.5-1.5B-Instruct ==========
---------- Configuraciones: ----------
	- Nombre del modelo generador: Qwen/Qwen2.5-1.5B-Instruct
	- Nombre del modelo de embeddings: paraphrase-multilingual-mpnet-base-v2
	- Especificaciones del modelo generador: {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
	- Especificaciones de los chunks: {'len_chunks': 300, 'remember_words': 100}
	- Especificaciones de la busqueda de similitud: {'coseno': True, 'top_k': 10}
-------------------------
Pregunta hecha al modelo: ¿Para que sirve el comando "cd"? ¿Puedes poner un ejemplo? Responde en menos de 50 palabras
Respuesta: Comando `cd`: Sirve para cambiar de directorio actual. Por ejemplo:

```bash
cd /
```

Cambiará la carpeta de trabajo al directorio raíz del sistema operativo.

```bash
cd Documents/
```

Saldrá a Documents dentro del directorio donde se encuentra ejecutando el script.

En general, el `cd` es utilizado para moverse hacia adelante y a

- Para la similitud euclidea:

In [24]:
# Generamos los parametros de las clases
params_generador = {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}  # Preferimos al final no moverlo porque funciona bastante bien
params_chunks = {'len_chunks': 500, 'remember_words': 200}
params_similitud = {'coseno': False, 'top_k': 20}

# Instanciamos la clase de pruebas
prueba_codigo_euclidea = GeneradorRespuestas(qwen, nombre_qwen, mpnet, nombre_mpnet, pregunta_codigo, documento_bash, params_generador, params_chunks, params_similitud)

# Mostramos el resultado
prueba_codigo_euclidea.generar_respueta()

========== Resultados con Qwen/Qwen2.5-1.5B-Instruct ==========
---------- Configuraciones: ----------
	- Nombre del modelo generador: Qwen/Qwen2.5-1.5B-Instruct
	- Nombre del modelo de embeddings: paraphrase-multilingual-mpnet-base-v2
	- Especificaciones del modelo generador: {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
	- Especificaciones de los chunks: {'len_chunks': 500, 'remember_words': 200}
	- Especificaciones de la busqueda de similitud: {'coseno': False, 'top_k': 20}
-------------------------
Pregunta hecha al modelo: ¿Para que sirve el comando "cd"? ¿Puedes poner un ejemplo? Responde en menos de 50 palabras
Respuesta: Respuesta:

`cd` se utiliza para cambiar de directorio actual, similar al comando `change directory`. Ejemplo:

```bash
cd Documents
```

Con este comando, se mueve al directorio Documents del usuario.


Vemos en los resultados, que no hay diferencias prácticamente. Aún así, nos quedaremos con la coseno porque **tuvo mejores resultados sin necesidad de usar muchos tokens, y responde bien a las preguntas 'especializadas.** Solo por eso la elegimos, sino, nos daría igual.

##### 3.3.1.6. Conclusiones finales para modelos 'normales'
Vimos que, de momento, funciona un poquito mejor en general la similitud coseno. Después de probar no solo cambiando el formato de los chunks, sino de cambiar las preguntas y las similitudes, que la distancia coseno es algo más fiable y directa que la euclidea. Esta última no lo hace mal, es otra opción para quien quiera variar.

##### Nota: Reinicia el kernel si ta dá errores de memoria el siguiente modelo

#### 3.3.2. Pruenas para jina y phi (los modelos especializados)
Ahora, vamos a ver si para modelos entrenados con código, son capaces de superar las limitaciones de los modelos 'normales':

In [8]:
# Preguntas con la cual comenzaremos
pregunta_normal = '¿Que es un interprete? ¿Cuales son sus funcionalidades basicas? Responde en menos de 50 palabras'
pregunta_codigo = '¿Para que sirve el comando "cd"? ¿Puedes poner un ejemplo? Responde en menos de 50 palabras'

##### 3.3.2.1. Probando con similitud coseno, con jina (lenguaje español 'especializado' en código), con chunk de tamaño 300, remember de 100 y top_k=10
Parecido a qwen, esta será la prueba estándar:

In [ ]:
# Generamos los parametros de las clases
params_generador = {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
params_chunks = {'len_chunks': 300, 'remember_words': 100}
params_similitud = {'coseno': True, 'top_k': 7}  # No podemos poner más porque sino explota nuestra gpu :,(

# Instanciamos la clase de pruebas
prueba_uno = GeneradorRespuestas(phi, nombre_phi, jina, nombre_jina, pregunta_normal, documento_bash, params_generador, params_chunks, params_similitud)

# Mostramos el resultado
prueba_uno.generar_respueta()

/tmp/ipykernel_246058/3760732647.py:55: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  producto_punto = np.dot(v1, v2)


========== Resultados con microsoft/Phi-3.5-mini-instruct ==========
---------- Configuraciones: ----------
	- Nombre del modelo generador: microsoft/Phi-3.5-mini-instruct
	- Nombre del modelo de embeddings: jinaai/jina-embeddings-v2-base-es
	- Especificaciones del modelo generador: {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
	- Especificaciones de los chunks: {'len_chunks': 300, 'remember_words': 100}
	- Especificaciones de la busqueda de similitud: {'coseno': True, 'top_k': 8}
-------------------------
Pregunta hecha al modelo: ¿Que es un interprete? ¿Cuales son sus funcionalidades basicas? Responde en menos de 50 palabras
Respuesta: Respuesta: Un interprete es un programa que ejecuta comandos de lenguaje de máquina o macro. En Bash, sus funcionalidades básicas incluyen ejecución interactiva y no interactiva de comandos, control de tareas, historial de ejecuciones y expandición de alias, según las directrices posix. 
[Fuerte]: El contexto indica que un 

Miremos el prompt:

In [11]:
# prueba_uno.mostrar_prompt()

Similar a qwen, lo hace decente (fallando en dar el chunk de donde sacó la información. Como que lo quiere dar, pero lo da mal). Le damos un 8.5/10.

##### 3.3.2.2. Probando con similitud euclidea, con jina (lenguaje español 'especializado' en código), con chunk de tamaño 300, remember de 100 y top_k=10
Vamos a ver como le va con la distancia euclidea:

In [17]:
# Generamos los parametros de las clases
params_generador = {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
params_chunks = {'len_chunks': 300, 'remember_words': 100}
params_similitud = {'coseno': False, 'top_k': 7}  # No podemos poner más porque sino explota nuestra gpu :,(

# Instanciamos la clase de pruebas
prueba_dos = GeneradorRespuestas(phi, nombre_phi, jina, nombre_jina, pregunta_normal, documento_bash, params_generador, params_chunks, params_similitud)

# Mostramos el resultado
prueba_dos.generar_respueta()

========== Resultados con microsoft/Phi-3.5-mini-instruct ==========
---------- Configuraciones: ----------
	- Nombre del modelo generador: microsoft/Phi-3.5-mini-instruct
	- Nombre del modelo de embeddings: jinaai/jina-embeddings-v2-base-es
	- Especificaciones del modelo generador: {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
	- Especificaciones de los chunks: {'len_chunks': 300, 'remember_words': 100}
	- Especificaciones de la busqueda de similitud: {'coseno': False, 'top_k': 7}
-------------------------
Pregunta hecha al modelo: ¿Que es un interprete? ¿Cuales son sus funcionalidades basicas? Responde en menos de 50 palabras
Respuesta: Respuesta: Un interprete es un programa que ejecuta comandos y scripts letra por letra, conectando salida e entrada. En Bash, funcionalidades interactivas incluyen control de tareas, edición en línea, historial y aliases.

Contexto:
- Capítulo 3: Funcionalidades Básicas del Intérprete (Sección 3.1)
- Capítulo 3: Sintaxis

Miremos el prompt:

In [ ]:
# prueba_dos.mostrar_prompt()

Lo hace bien, pero no nos gusta que no nos de el número de los fragmentos de los chunks: Analiza tanto que nos da el contexto de los índices. Un 7/10 solo porque responde muy bien.

##### 3.3.2.3. Omitimos la pruba con cambios en los chunks
Porque no nos da la GPU.

##### 3.3.2.4. Conclusiones pregunta normal
Luego de probar los 2 modelos restantes, vemos que no hay mucha diferencia con la pregunta normal, a excepción de la euclidea, al darnos contextos que no le dijimos. Vamos a ver la prueba de fuego ahora con la pregunta 'especializada':

- Para la similitud coseno:

In [21]:
# Generamos los parametros de las clases
params_generador = {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
params_chunks = {'len_chunks': 300, 'remember_words': 100}
params_similitud = {'coseno': True, 'top_k': 7}  # No podemos poner más porque sino explota nuestra gpu :,(

# Instanciamos la clase de pruebas
prueba_codigo_coseno = GeneradorRespuestas(phi, nombre_phi, jina, nombre_jina, pregunta_codigo, documento_bash, params_generador, params_chunks, params_similitud)

# Mostramos el resultado
prueba_codigo_coseno.generar_respueta()

/tmp/ipykernel_246058/3760732647.py:55: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  producto_punto = np.dot(v1, v2)


========== Resultados con microsoft/Phi-3.5-mini-instruct ==========
---------- Configuraciones: ----------
	- Nombre del modelo generador: microsoft/Phi-3.5-mini-instruct
	- Nombre del modelo de embeddings: jinaai/jina-embeddings-v2-base-es
	- Especificaciones del modelo generador: {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
	- Especificaciones de los chunks: {'len_chunks': 300, 'remember_words': 100}
	- Especificaciones de la busqueda de similitud: {'coseno': True, 'top_k': 7}
-------------------------
Pregunta hecha al modelo: ¿Para que sirve el comando "cd"? ¿Puedes poner un ejemplo? Responde en menos de 50 palabras
Respuesta: Output: El comando "cd" cambia el directorio de trabajo actual. Ejemplo: `cd /usr/bin` Cambia el directorio de trabajo actual al directorio /usr/bin. 

Contexto:
208.- es llamado sin opciones), empe- zando por cero.-N Muestra el directorio número N (contando desde la derecha de la lista imprimida por dirs cuando es llamado sin 

- Para la similitud euclidea:

In [23]:
# Generamos los parametros de las clases
params_generador = {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
params_chunks = {'len_chunks': 300, 'remember_words': 100}
params_similitud = {'coseno': False, 'top_k': 7}  # No podemos poner más porque sino explota nuestra gpu :,(

# Instanciamos la clase de pruebas
prueba_codigo_euclidea = GeneradorRespuestas(phi, nombre_phi, jina, nombre_jina, pregunta_codigo, documento_bash, params_generador, params_chunks, params_similitud)

# Mostramos el resultado
prueba_codigo_euclidea.generar_respueta()

========== Resultados con microsoft/Phi-3.5-mini-instruct ==========
---------- Configuraciones: ----------
	- Nombre del modelo generador: microsoft/Phi-3.5-mini-instruct
	- Nombre del modelo de embeddings: jinaai/jina-embeddings-v2-base-es
	- Especificaciones del modelo generador: {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
	- Especificaciones de los chunks: {'len_chunks': 300, 'remember_words': 100}
	- Especificaciones de la busqueda de similitud: {'coseno': False, 'top_k': 7}
-------------------------
Pregunta hecha al modelo: ¿Para que sirve el comando "cd"? ¿Puedes poner un ejemplo? Responde en menos de 50 palabras
Respuesta: Output: El comando "cd" cambia el directorio actual. Ejemplo: Si quieres cambiar de directorio de usuario al directorio Documentos, ejecuta `cd Documentos`.
Output en formato de referencia: [208.-] Capı́tulo 6: Funcionalidades de Bash 103 dir


Lo hace bien, se explica bien y se entiende, y se da el contexto para la distancia coseno. Para la euclidea, también es correcto.

##### 3.3.2.5. Conclusiones finales para modelos 'especializados'
En general, lo hizo bastante bien con ambas distancias. Es verdad que la euclidea al final, no llega a decir exáctamente que fragmento o chunk usó para sacar la información, pero en general, ambas funcionan muy bien. Solo por ese detalle, nos quedamos (de nuevo) con la similitud coseno.

#### 3.4. Conclusiones finales
Ambos modelos lo hicieron bien. A lo mejor, el modelo 'normal' (qwen y mpnet) destacaron algo mejor en las preguntas normales (que no llevavan código), mientras que los modelos 'especiales' (jina y phi) lo hicieron algo mejor en código. Las diferencias no son notorias, y la segunda pareja (la especial), no logró demostrar todo el potencial al tener que cuantizar los modelos. Aún así, es un punto de desventaja para esta pareja, lo que hace inclinar la balanza algo a favor de los modelos 'normales': Qwen y MPNET.

## 4. Probando las mejores opciones con más preguntas
Ahora, solo queda probar más preguntas y ver que tal responde ambos modelos para sacar conclusiones:

### 4.1. Preguntas
Definimos las preguntas que le haremos a los modelos:

In [10]:
# Definimos las preguntas
preguntas = [
    "¿Que hace el comando grep en Bash?",
    "¿Para que se utiliza awk en procesamiento de texto?",
    "¿Como funciona el comando sed para reemplazar cadenas?",
    "¿Cuales son las caracteristicas de BASH?",
    "¿Cual es la diferencia entre grep y awk?",
    "¿Para qué sirve el comando umask en Bash?"
]

### 4.2. Probamos a la pareja 'normal'
Vamos a ver que tal le va con este banco de preguntas (usando la mejor configuración: la prueba uno):

In [10]:
# Generamos los parametros de las clases
params_generador = {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
params_chunks = {'len_chunks': 300, 'remember_words': 100}
params_similitud = {'coseno': True, 'top_k': 10}

In [12]:
# Generamos la respuesta con cada pregunta
for pregunta in preguntas:

    # Indicamos que es respuesta a una pregunta
    print('█' * 10, f'RESPUESTA A LA PREGUNA "{pregunta}"', '█' * 10)

    # Instanciamos la clase de pruebas
    generador_respuesta_normal = GeneradorRespuestas(qwen, nombre_qwen, mpnet, nombre_mpnet, pregunta, documento_bash,
                                                        params_generador, params_chunks, params_similitud)

    # Mostramos el resultado
    generador_respuesta_normal.generar_respueta()

██████████ RESPUESTA A LA PREGUNA "¿Que hace el comando grep en Bash?" ██████████
========== Resultados con Qwen/Qwen2.5-1.5B-Instruct ==========
---------- Configuraciones: ----------
	- Nombre del modelo generador: Qwen/Qwen2.5-1.5B-Instruct
	- Nombre del modelo de embeddings: paraphrase-multilingual-mpnet-base-v2
	- Especificaciones del modelo generador: {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
	- Especificaciones de los chunks: {'len_chunks': 300, 'remember_words': 100}
	- Especificaciones de la busqueda de similitud: {'coseno': True, 'top_k': 10}
-------------------------
Pregunta hecha al modelo: ¿Que hace el comando grep en Bash?
Respuesta: Responda:

El comando 'grep' en Bash ejecuta un búsqueda de texto basada en expresiones regulares. Utiliza la opción "-n", que muestra el número de línea asociada con la ocurrencia de la regla encontrada. Además, utiliza la opción '-i', que no hace diferencia entre mayúsculas y minúsculas durante el proceso d

Muy buenas respuestas. Es verdad que algunas no dice de donde sacó el contexto, pero nos sentimos alegres que, en la pregunta que dice no encontrar información, no alucinó y dijo que no encuentra esa información:
- 'No encuentro esa información en los fragmentos'. 

### 4.3. Probamos a la pareja 'especializada'
Vamos a ver que tal le va con este banco de preguntas:

In [5]:
# Generamos los parametros de las clases
params_generador = {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
params_chunks = {'len_chunks': 300, 'remember_words': 100}
params_similitud = {'coseno': True, 'top_k': 7}  # No podemos poner más porque sino explota nuestra gpu :,(

In [13]:
# Generamos la respuesta con cada pregunta
for pregunta in preguntas:

    # Indicamos que es respuesta a una pregunta
    print('█' * 10, f'RESPUESTA A LA PREGUNA "{pregunta}"', '█' * 10)

    # Instanciamos la clase de pruebas
    generador_respuesta_especial = GeneradorRespuestas(phi, nombre_phi, jina, nombre_jina, pregunta, documento_bash,
                                                        params_generador, params_chunks, params_similitud)

    # Mostramos el resultado
    generador_respuesta_especial.generar_respueta()

██████████ RESPUESTA A LA PREGUNA "¿Que hace el comando grep en Bash?" ██████████


/tmp/ipykernel_257307/3760732647.py:55: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  producto_punto = np.dot(v1, v2)


========== Resultados con microsoft/Phi-3.5-mini-instruct ==========
---------- Configuraciones: ----------
	- Nombre del modelo generador: microsoft/Phi-3.5-mini-instruct
	- Nombre del modelo de embeddings: jinaai/jina-embeddings-v2-base-es
	- Especificaciones del modelo generador: {'max_new_tokens': 250, 'do_sample': True, 'temperature': 1, 'top_p': 0.9}
	- Especificaciones de los chunks: {'len_chunks': 300, 'remember_words': 100}
	- Especificaciones de la busqueda de similitud: {'coseno': True, 'top_k': 7}
-------------------------
Pregunta hecha al modelo: ¿Que hace el comando grep en Bash?
Respuesta: Respuesta:
El comando `grep` en Bash se utiliza para buscar texto o patrones específicos dentro de archivos o entrada de flujo estándar. No daña ni modifica ningún archivo o entrada.

Con respecto a las capacidades y opciones detalladas en el contexto proporcionado:

- `--help` muestra un mensaje de uso y finaliza el comando, lo que indica que `grep -h` o `grep --help` proporciona ayu

Respuestas buenas, aunque sorprende que para algunas, este dice que no hay información en el informe, mientras que los modelos normales sí lo hacian. Quitando eso, pasa lo contrario, así que no está mal tampoco.

### 4.4. Conclusiones y resultados
RLas respuestas a las preguntas fueron mayormente de calidad, donde algunas lamentablemente se saltaron la parte de mencionar **de qué fragmentos sacan la respuesta.** Quitando eso, vimos una discrepancia entre ambos modelos, donde uno dice que no hay información en los modelos para responder, pero el otro decía lo contrario y respondia (en verdad, hay información para todo. Es verdad que para algunos como grep o umask, es algo escasa, pero hay). Nos dimos cuenta al final, que aunque el de código entienda y sea perfecto para analizar código en general, a veces, con entender solo el contexto de la pregunta basta para poder responder las preguntas. Si nos tenemos que quedar con un ganador, nos quedamos con lo bueno, bonito y barato: La pareja 'normal' (no nos gusta tryharderas :D)

# 5. Conclusiones finales
Ha sido muy interesante este método RAG. Si para estos modelos, que encima, alguno hemos tenido que limitarlo, ya respondían bastante bien, implementar esto con modelos como Gemini, ChatGPT o similares, debe de ser una barbaridad (es más, hay chatbots que evolucionan a esto en la vida real). No olvidemos que, esto incluye también prompt engineering, ya que si hacemos un buen prompt, los modelos trabajarán y darán mejores respuestas. En general, como continuación del Notebook de LoRA, estas técnicas de usos avanzados de modelos transformers ha sido impresionante y se nota, a la hora de la verdad, que termina siendo muy útil (en especial para los mortales como nosotros).